In [1]:
%matplotlib inline
%config Completer.use_jedi=False

In [2]:
import glob

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
import matplotlib.backends.backend_pdf

from tqdm import tqdm_notebook
from collections import deque

/usr/local/Cellar/python3/3.5.2_1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
plt.rcParams['figure.figsize'] = (20, 10)

## External Arrivals

#### Plotting Function

In [4]:
def plot_hist(ext, ext_ta, title, pdf):
    fig = plt.figure()
    
    bins_tmp = list(np.arange(len(list(set(ext)))))
    bins_tmp = bins_tmp + [bins_tmp[-1] + 1]
    counts, bins, _, = plt.hist([ext, ext_ta], bins=bins_tmp, align='left', 
                            color=['b','r'], alpha=0.5, label=['External Arrivals', 'External Arrivals Turned Away'])
    
    #label bars
    for i in range(len(bins_tmp) - 1):
        label_ext, label_ta = len([j for j in ext if j == bins_tmp[i]]), len([j for j in ext_ta if j == bins_tmp[i]])
        
        plt.text(bins_tmp[i] - 0.19, label_ext, str(label_ext), size=7, ha='center', va='bottom')
        plt.text(bins_tmp[i] + 0.205, label_ta, str(label_ta), size=7, ha='center', va='bottom')
    
    
    plt.legend()
    plt.ylim(0, max(counts[0].tolist() + counts[1].tolist()) + 100)
    plt.title(title)
    plt.xlabel('Number Arrivals/Turned Away')
    plt.ylabel('Frequency of Occurence')
    
    pdf.savefig(fig)
    
    df = pd.DataFrame()
    df['external'] = counts[0]
    df['turned_away'] = counts[1]
    
    return df

#### Loop through all files and plot into on pdf per beta
* Load ICU size file.
* Change high or low manually.

In [6]:
df_cap = pd.read_excel('../../data/summary_extern_arrival_load_computation.xlsx', header=2)
df_cap = df_cap.dropna()
df_cap.HOSP = df_cap.HOSP.astype(int)
df_cap = df_cap[['HOSP', 'N_ICU']]
df_cap

,HOSP,N_ICU
0,15,6
1,22,9
2,27,9
3,8,11
4,10,11
5,21,11
6,5,12
7,14,12
8,11,16
9,17,16


***

In [6]:
#state_files = sorted([f for f in glob.glob('../../data/high/*states*.txt')],
#                     key=lambda x: int(x.split('/')[-1].split('_')[1]))

In [7]:
#pdf = matplotlib.backends.backend_pdf.PdfPages('../../data/plots/high_hosp_ext_ta.pdf')

#list_of_dfs = []
#for f in state_files:
#    icu_size = df_cap.loc[df_cap.HOSP == int(f.split('/')[-1].split('_')[1])].N_ICU.values[0]
    
#    ext, ext_ta = [], []
#    f_tmp = open(f).readlines()
#    for s in f_tmp:
#        s = s.strip().split()
#        ext.append(int(s[-1]))
#        ext_ta.append(int(s[-1]) - (icu_size - int(s[1])))
    
#    df = plot_hist(ext, ext_ta, ': '.join(f.split('/')[-1].split('_')[:2]) + ' (ICU Size: ' + str(icu_size) + ')', pdf)
#    df.to_csv('../../data/csvs/high/' + '_'.join(f.split('/')[-1].split('_')[:2]) + '.csv')
#pdf.close()

## Waiting Time and Rerouting Probability

#### Descriptive stats functions

In [8]:
def get_ss(states, actions, icu_beds):
    obs = {}

    #10, 11
    avg_patients_ICU = []
    avg_patients_ED = []

    #avg number of patients rerouted that are low, high
    #reroute / (reroute + admit total)
    reroute_prob = 0
    
    total_patients = 0

    state_txt = [i.replace('\n', '').strip() for i in states]
    action_txt = [i.replace('\n', '').strip() for i in actions]

    for l in range(len(state_txt)):
        state = [int(z) for z in state_txt[l].split()]
        action = [int(z) for z in action_txt[l].split()]
    
        avg_patients_ICU.append(state[1])
        
        #patients in ED AFTER action is taken
        avg_patients_ED.append((state[0] - action[0] - action[1]))

        reroute_prob += action[1]
        total_patients += action[0] + action[1]
            
    obs['reroute_prob'] = float(reroute_prob) / (total_patients)
    
    obs['avg_patients_ICU'] = np.around(np.nanmean(avg_patients_ICU), decimals=2)
    obs['avg_patients_ED'] = np.around(np.nanmean(avg_patients_ED), decimals=5)
    
    return obs

In [9]:
def get_wait(states, actions):
    wts = {}
    
    #create a stack of unique identifiers (LIFO)
    uniq = 1

    low_wt = []

    state_txt = [i.replace('\n', '').strip() for i in states]
    action_txt = [i.replace('\n', '').strip() for i in actions]

    #create a queue to hold low and high patients and time of decision
    low_queue = deque([])

    timer = 0
    left_in_ED_low = 0
    
    start = True
    
    for i in tqdm_notebook(range(len(state_txt))):
        state = state_txt[i].split()
        action = action_txt[i].split()

        #add to queue at time X
        if start == True:
            for j in range(int(state[0])):
                low_queue.append((uniq, timer))
                uniq += 1
            start = False
            
        elif left_in_ED_low > 0 and start == False:
            new_low = np.abs(left_in_ED_low - int(state[0]))
            #print(new_low, new_high)

            for j in range(new_low):
                low_queue.append((uniq, timer))
                uniq += 1
                    
        elif left_in_ED_low == 0:
            for j in range(int(state[0])):
                low_queue.append((uniq, timer))
                uniq += 1

        #print(low_queue, high_queue, '\n')

        admit_low, reroute_low = int(action[0]), int(action[1])
        
        #remove from queue at time X
        print(i, int(state[0]), int(action[0]), int(action[1]), left_in_ED_low, len(low_queue))
        for m in range(admit_low):
            popped = low_queue.popleft()
            low_wt.append((timer - popped[1]))
        
        
        for m in range(reroute_low):
            popped = low_queue.popleft()
            low_wt.append((timer - popped[1]))
            
        left_in_ED_low = len(low_queue)

        timer += 1

        #print(low_queue, high_queue)
        #print(left_in_ED_low, left_in_ED_high, left_in_ED, '\n')
        
        
    
    return (np.nanmean(low_wt), np.nanstd(low_wt))

In [10]:
state_files = sorted([f for f in glob.glob('../../data/high/*states*.txt')],
                     key=lambda x: int(x.split('/')[-1].split('_')[1]))
action_files = sorted([f for f in glob.glob('../../data/high/*actions*.txt')],
                     key=lambda x: int(x.split('/')[-1].split('_')[1]))

In [11]:
#pdf = matplotlib.backends.backend_pdf.PdfPages('../../plots/wait_and_reroute.pdf')
for fi in tqdm_notebook(range(len(state_files))):
    print(state_files[fi], action_files[fi])
    
    
    tmps = [' '.join(s.strip().split()[:2]) for s in open(state_files[fi], 'r').readlines()]
    tmpa = open(action_files[fi], 'r').readlines()

    tups = get_wait(tmps, tmpa)
    #ss = get_ss(tmps, tmpa, int(p.split('_')[-1]))

    #average stage cost is calculated within c++ file
    #ss['avg_wait_time'] = tups[0]

    #wts[p] = {'waiting_time' : ss['avg_wait_time'], 'reroute_probability' : ss['reroute_prob']}

#df_ss = pd.DataFrame(wts).T
#df_ss = df_ss[['waiting_time', 'reroute_probability']]
#df_ss.to_excel('../../plots/wait_and_reroute_' + b + '.xlsx')

#ax = df_ss.plot(kind='bar')
#plt.title('Beta: ' + b)
#plt.ylabel('Wait Time/Reroute Probability')
#plt.xlabel('Hospital Types')
#plt.tight_layout()
#fig = ax.get_figure()
#pdf.savefig(fig)

#pdf.close()

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

../../data/high/HOSP_2_observed_states.txt ../../data/high/HOSP_2_observed_actions.txt


HBox(children=(IntProgress(value=0, max=2113), HTML(value='')))

0 1 1 0 0 1
1 1 0 1 0 1
2 0 0 0 0 0
3 1 1 0 0 1
4 0 0 0 0 0
5 0 0 0 0 0
6 0 0 0 0 0
7 0 0 0 0 0
8 0 0 0 0 0
9 0 0 0 0 0
10 0 0 0 0 0
11 0 0 0 0 0
12 0 0 0 0 0
13 0 0 0 0 0
14 0 0 0 0 0
15 0 0 0 0 0
16 0 0 0 0 0
17 0 0 0 0 0
18 0 0 0 0 0
19 0 0 0 0 0
20 0 0 0 0 0
21 0 0 0 0 0
22 1 1 0 0 1
23 1 0 1 0 1
24 0 0 0 0 0
25 0 0 0 0 0
26 0 0 0 0 0
27 0 0 0 0 0
28 0 0 0 0 0
29 0 0 0 0 0
30 0 0 0 0 0
31 1 1 0 0 1
32 0 0 0 0 0
33 0 0 0 0 0
34 1 1 0 0 1
35 0 0 0 0 0
36 0 0 0 0 0
37 0 0 0 0 0
38 1 1 0 0 1
39 1 0 0 0 1
40 1 0 0 1 1
41 1 0 0 1 1
42 1 0 0 1 1
43 1 0 0 1 1
44 1 0 1 1 1
45 0 0 0 0 0
46 1 1 0 0 1
47 1 0 1 0 1
48 0 0 0 0 0
49 0 0 0 0 0
50 0 0 0 0 0
51 2 1 0 0 2
52 1 0 0 1 1
53 1 0 0 1 1
54 1 0 0 1 1
55 1 0 0 1 1
56 1 0 0 1 1
57 1 0 0 1 1
58 1 0 0 1 1
59 1 0 1 1 1
60 0 0 0 0 0
61 0 0 0 0 0
62 0 0 0 0 0
63 0 0 0 0 0
64 0 0 0 0 0
65 1 0 1 0 1
66 1 1 0 0 1
67 0 0 0 0 0
68 0 0 0 0 0
69 0 0 0 0 0
70 0 0 0 0 0
71 0 0 0 0 0
72 0 0 0 0 0
73 0 0 0 0 0
74 1 1 0 0 1
75 0 0 0 0 0
76 0 0 0 0 0
77 0 0 0 

733 0 0 0 0 0
734 0 0 0 0 0
735 0 0 0 0 0
736 0 0 0 0 0
737 0 0 0 0 0
738 1 0 1 0 1
739 1 0 1 0 1
740 1 1 0 0 1
741 0 0 0 0 0
742 0 0 0 0 0
743 0 0 0 0 0
744 0 0 0 0 0
745 0 0 0 0 0
746 0 0 0 0 0
747 0 0 0 0 0
748 0 0 0 0 0
749 0 0 0 0 0
750 0 0 0 0 0
751 1 0 1 0 1
752 0 0 0 0 0
753 0 0 0 0 0
754 1 1 0 0 1
755 1 0 1 0 1
756 0 0 0 0 0
757 0 0 0 0 0
758 0 0 0 0 0
759 0 0 0 0 0
760 0 0 0 0 0
761 0 0 0 0 0
762 0 0 0 0 0
763 1 1 0 0 1
764 0 0 0 0 0
765 0 0 0 0 0
766 0 0 0 0 0
767 1 0 1 0 1
768 1 1 0 0 1
769 0 0 0 0 0
770 0 0 0 0 0
771 0 0 0 0 0
772 0 0 0 0 0
773 0 0 0 0 0
774 0 0 0 0 0
775 0 0 0 0 0
776 0 0 0 0 0
777 1 1 0 0 1
778 0 0 0 0 0
779 1 0 1 0 1
780 0 0 0 0 0
781 0 0 0 0 0
782 0 0 0 0 0
783 1 0 1 0 1
784 0 0 0 0 0
785 0 0 0 0 0
786 0 0 0 0 0
787 0 0 0 0 0
788 1 0 1 0 1
789 0 0 0 0 0
790 0 0 0 0 0
791 0 0 0 0 0
792 0 0 0 0 0
793 0 0 0 0 0
794 0 0 0 0 0
795 0 0 0 0 0
796 0 0 0 0 0
797 0 0 0 0 0
798 0 0 0 0 0
799 1 1 0 0 1
800 0 0 0 0 0
801 1 1 0 0 1
802 0 0 0 0 0
803 0 0 0 0 0
804 0 

1322 1 0 1 323575 647149
1323 0 0 0 647148 1294296
1324 0 0 0 1294296 2588592



KeyboardInterrupt: 